Alex Malz, David Mykytyn

In [ ]:
from collections import namedtuple
import numpy as np
import scipy.stats as sps
import scipy.optimize as spo
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
LC = namedtuple('LC', ('x', 'y'))

 may need to preprocess to keep it reasonable, constraints on delta/stretch

# Simulate some mock data

In [ ]:
def make_gauss(loc, scale, amp=1., const=0.):
    func = sps.norm(loc, scale)
    out = lambda x: amp * func.pdf(x) + const
    return out

def make_sine(period, phase=0., amp=1., const=0.):
    func = lambda x: amp * (np.sin(period * x + phase)) + const
    return func

In [ ]:
def make_cadence(x, scatter):
    assert(np.all((x[1:]-x[:-1]) > scatter))
    jitter = (np.random.uniform(np.shape(x)) - 0.5) * scatter * 2.
    perturbed = x + jitter
    return perturbed

def noisify_obs(y, scatter):
    errs = scatter * np.ones_like(y)
    new_y = y + sps.norm(0., scatter).rvs(np.shape(y))
    return(new_y, errs)

In [ ]:
def_cadence = np.arange(0., 200., 5.)

gmodel = make_gauss(100., 10., 50., 1.)
gtimes = make_cadence(def_cadence, 0.5)
gphot, gerr = noisify_obs(gmodel(gtimes), 0.1)
glc = LC(gtimes, gphot)

smodel = make_sine(20., 0., 5., 5.)
stimes = make_cadence(def_cadence, 0.5)
sphot, serr = noisify_obs(smodel(stimes), 0.3)
slc = LC(stimes, sphot)

In [ ]:
plt.errorbar(glc.x, glc.y, yerr=gerr, linestyle='None', marker='o')
plt.errorbar(slc.x, slc.y, yerr=serr, linestyle='None', marker='+')

## Permitted transformations

* shiftx
* stretchx
* shifty
* stretchy
* (cross-talk between bands)


In [ ]:
def transform(lc, deltax, deltay, stretchx, stretchy):
    new_x = (stretchx * lc.x) + deltax
    new_y = (stretchy * lc.y) + deltay
    return LC(new_x, new_y)

def merge(lca, lcb):
    new_x = np.concatenate((lca.x, lcb.x))
    new_y = np.concatenate((lca.y, lcb.y))
    order = np.argsort(new_x)
    ord_x = new_x[order]
    ord_y = new_y[order]
    return LC(ord_x, ord_y)

# Reduce to summary statistics (consistency metric)

Contenders:

* periodogram -- identify periodicity and stochastic noise levels
* flux per time bins -- trends keeping bin size constant but changing bin ends
* abs/percent change in color and total flux/magnitude

find MAP/MLE of p(A = B | lc_A, lc_B)
optimize over shift/stretch params

merge (x_A, x_B) and (y_A, y_B)

In [ ]:
def connect_the_dots(lc):
    x_difs = lc.x[1:] - lc.x[:-1]
    y_difs = lc.y[1:] - lc.y[:-1]
    sol = np.sqrt(x_difs ** 2 + y_difs **2)
    return np.sum(sol)

In [ ]:
def find_max_prob(lca, lcb):
    
    origa = connect_the_dots(lca)
    difmax = np.max((lca.x[-1]-lca.x[0], lcb.x[-1]-lcb.x[0]))
    difmin = np.min((np.min(lca.x[1:]-lca.x[:-1]), np.min(lcb.x[-1]-lcb.x[0])))
    
    
    def dlim_lo(params):
        return params[:2]
    def dlim_hi(params):
        return difmax - params[:2]
    def slim_hi(params):
        return difmax - params[2:]
    def slim_lo(params):
        return params[2:] - difmin
    def slim(params):
        return params[2:]
    
    constraints = [dlim_hi, dlim_lo, slim_hi, slim_lo, slim]
    
    debug = []
    def _helper(params):
        (deltax, deltay, stretchx, stretchy) = params
        lc = transform(lcb, deltax, deltay, stretchx, stretchy)
        new_len = connect_the_dots(lc)
        lc_both = merge(lca, lc) 
        length = connect_the_dots(lc_both) / stretchy * stretchx
        to_min = length - origa - new_len
        debug.append(to_min)
        return(to_min)
    
    ivals = (0., 0., 1., 1.)
    res = spo.fmin_cobyla(_helper, ivals, constraints)
    return(res, debug)



In [ ]:
# gtimes2 = gtimes + 50. * np.ones_like(gtimes)#ake_cadence(def_cadence, 0.5)
# gphot2, gerr2 = gphot, gerr#noisify_obs(gmodel(gtimes2), 0.1)
# glc2 = LC(gtimes2, gphot2)
glc2 = transform(glc, 50., 0., 1., 1.)
plt.plot(glc.x, glc.y)
plt.plot(glc2.x, glc2.y)
print((connect_the_dots(glc), connect_the_dots(glc2), connect_the_dots(merge(glc, glc2))))

In [ ]:
ans, debug = find_max_prob(glc, glc2)
print(ans)
plt.plot(debug)

In [ ]:
(dx, dy, sx, sy) = ans
fin = transform(glc2, dx, dy, sx, sy)
print(connect_the_dots(fin))
plt.plot(glc.x, glc.y)
plt.plot(fin.x, fin.y)

In [ ]:
help(spo.minimize)

# Cluster in the space of summary statistics

kdtree (and more)

# Other ideas

pairwise combinations/comparisons?